In [1]:
# Direct lammps script implemetation in pyiron 
from pyiron import Project
import numpy as np
import pandas
from jinja2 import Template
import matplotlib.pyplot as plt 
import scipy.constants as sc
from scipy.integrate import cumtrapz
import os

# Project 
pr_1 = Project("/nfshome/deshmukh/pyiron/projects/NASICON/project/hena/hena_2_2/minimization_small")           #! Path for postprocessing 
pr = Project("/nfshome/deshmukh/pyiron/projects/NASICON/project/hena/hena_2_2/glass_small")  

In [2]:
# Calculate quech rate and time 
rate = range(1, 2, 1)
#rate_1 = range(1, 11, 1)

t = []     # time steps in fs 
for i in rate:
    time = ((4000-300)/i)*1000
    t.append(round(time))

# For different colling rate 

strct_no = [82,139,34]#,23,26]#, 106,  13,  15,  57,  67,  66] 
temp = [523, 573, 623, 673, 723, 773]

#for k in strct_no:
# Crystal calculations 
    #job = pr['hena_1_%s_rate_1_k_ps'%k]
    #print(job.output.energy_pot[-1])
    #job.transfer_from_remote()
    #job.compress()

In [ ]:
en = []
for i in strct_no:
    job = pr['hena_1_struct_mini_%s'%i]
    atom = len(job.get_final_structure())
    en.append(job['output/generic/energy_tot'][-1]/atom)

In [6]:
for k in strct_no:
    for j,i in zip(temp,range(len(temp))):
        job_mini = pr['hena_1_%s_rate_1_k_ps'%k]               # Bulk Crystal
    
        # Get last structure    
        struct_nvt = job_mini.get_structure(iteration_step=-1) # Last frame from minimization
    
        j = pr.create.job.Vasp("hena_1_glass_small_%s_%sk"%(k,j), delete_existing_job=True)
        j.calc_md(temperature=temp[i], n_ionic_steps=25000, n_print=1, time_step=1.0)     # AIMD simulations        
        j.structure = struct_nvt                               # Intial structure   
        j.set_kpoints(scheme="GC")  
        j.input.kpoints.set_kpoints_file(size_of_mesh=['1 1 1'])
        j.set_exchange_correlation_functional("PBE")           # GGA (PBE) potential, also need to test SCAN, rSCAN, r2SCAN     

        #Select PAW potentials 
        #j.list_potentials()
        j.potential.Si = "Si"                                  # 4 valence electron, EMAX = 245 (eV)
        j.potential.P = "P"                                    # 5 valence electron, EMAX = 255 (eV) 
        j.potential.O = "O"                                    # 6 valence electron, EMAX = 400 (eV)   
        j.potential.Na = "Na"                                  # 7 valence electron with 'p' orbitals, EMAX = 260 (eV) 
        j.potential.Zr = "Zr_sv"                               # 12 valence electron with 's' orbitals, EMAX = 230 (eV)  

        # Edit INCAR file 
        j.input.incar["SMASS"] = 0                             # For noosessh host pyironak condition, accurate cal = 1e-8 (2) -- Conv 
        j.input.incar["ALGO"] = 'Normal'                       # For convergence use Fast else Normal
        # Paralellization 
        j.input.incar["ISYM"] = 0                              # no symmetry
        j.input.incar["MAXMIX"] = 40                           # reuse mixer from one MD step to next
        j.input.incar["NELMIN"] = 4                            # number of electronic steps
        j.input.incar["NCORE"] = 12                            # Core for calculations  
        #j.input.incar["KPAR"] = 12                            # No of K-points treated in parallel, must be divi to IBZKPT  
        j.server.cores = 192
        j.executable.executable_path = '/home/vd80naku/pyiron/resources/vasp/bin/run_vasp_6.1.2_default_mpi_gamma.sh'
        j.server.queue = "vasp_l_l2"
        j.run()

/tmp/ipykernel_39325/1823927583.py:6: DeprecationWarning: pyiron_atomistics.atomistics.structure.has_structure.get_structure(iteration_step=-1) is deprecated.
  struct_nvt = job_mini.get_structure(iteration_step=-1) # Last frame from minimization
2022-09-26 19:47:21,474 - pyiron_log - WARNING - Deprecated data structure! Returned array was converted from dtype='O' to dtype=float64 via `np.array(result.tolist())`.
Please run rewrite_hdf5() (from a job: job.project_hdf5.rewrite_hdf5() ) to update this data! To update all your data run Project.maintenance.update.base_v0_3_to_v0_4('all').
2022-09-26 19:47:22,256 - pyiron_log - WARNING - Deprecated data structure! Returned array was converted from dtype='O' to dtype=float64 via `np.array(result.tolist())`.
Please run rewrite_hdf5() (from a job: job.project_hdf5.rewrite_hdf5() ) to update this data! To update all your data run Project.maintenance.update.base_v0_3_to_v0_4('all').
2022-09-26 19:47:22,572 - pyiron_log - WARNING - Deprecated dat

The job hena_1_glass_small_82_523k was saved and received the ID: 13104


/nfshome/deshmukh/miniconda3/envs/vaibhav/lib/python3.9/site-packages/pysqa/utils/remote.py:235: UserWarning: 
  warnings.warn(stderr.read().decode())
100%|██████████| 4/4 [00:00<00:00, 186.91it/s]
/nfshome/deshmukh/miniconda3/envs/vaibhav/lib/python3.9/site-packages/pysqa/utils/remote.py:235: UserWarning: 2022-09-26 19:48:09,784 - pyiron_log - INFO - job: hena_1_glass_small_82_523k id: 6, status: initialized, submitted: queue id 31659942

  warnings.warn(stderr.read().decode())


Queue system id:  31659942


/tmp/ipykernel_39325/1823927583.py:6: DeprecationWarning: pyiron_atomistics.atomistics.structure.has_structure.get_structure(iteration_step=-1) is deprecated.
  struct_nvt = job_mini.get_structure(iteration_step=-1) # Last frame from minimization
2022-09-26 19:48:21,645 - pyiron_log - WARNING - Deprecated data structure! Returned array was converted from dtype='O' to dtype=float64 via `np.array(result.tolist())`.
Please run rewrite_hdf5() (from a job: job.project_hdf5.rewrite_hdf5() ) to update this data! To update all your data run Project.maintenance.update.base_v0_3_to_v0_4('all').
2022-09-26 19:48:22,299 - pyiron_log - WARNING - Deprecated data structure! Returned array was converted from dtype='O' to dtype=float64 via `np.array(result.tolist())`.
Please run rewrite_hdf5() (from a job: job.project_hdf5.rewrite_hdf5() ) to update this data! To update all your data run Project.maintenance.update.base_v0_3_to_v0_4('all').
2022-09-26 19:48:22,620 - pyiron_log - WARNING - Deprecated dat

...

/tmp/ipykernel_39325/1823927583.py:6: DeprecationWarning: pyiron_atomistics.atomistics.structure.has_structure.get_structure(iteration_step=-1) is deprecated.
  struct_nvt = job_mini.get_structure(iteration_step=-1) # Last frame from minimization
2022-09-26 20:10:34,683 - pyiron_log - WARNING - Deprecated data structure! Returned array was converted from dtype='O' to dtype=float64 via `np.array(result.tolist())`.
Please run rewrite_hdf5() (from a job: job.project_hdf5.rewrite_hdf5() ) to update this data! To update all your data run Project.maintenance.update.base_v0_3_to_v0_4('all').
2022-09-26 20:10:35,297 - pyiron_log - WARNING - Deprecated data structure! Returned array was converted from dtype='O' to dtype=float64 via `np.array(result.tolist())`.
Please run rewrite_hdf5() (from a job: job.project_hdf5.rewrite_hdf5() ) to update this data! To update all your data run Project.maintenance.update.base_v0_3_to_v0_4('all').
2022-09-26 20:10:35,589 - pyiron_log - WARNING - Deprecated dat

The job hena_1_glass_small_26_773k was saved and received the ID: 13138


/nfshome/deshmukh/miniconda3/envs/vaibhav/lib/python3.9/site-packages/pysqa/utils/remote.py:235: UserWarning: 
  warnings.warn(stderr.read().decode())
100%|██████████| 4/4 [00:00<00:00, 183.51it/s]


Queue system id:  31660069


/nfshome/deshmukh/miniconda3/envs/vaibhav/lib/python3.9/site-packages/pysqa/utils/remote.py:235: UserWarning: 2022-09-26 20:11:10,138 - pyiron_log - INFO - job: hena_1_glass_small_26_773k id: 23, status: initialized, submitted: queue id 31660069

  warnings.warn(stderr.read().decode())


In [ ]:
for k in strct_no:
    for j,i in zip(temp,range(len(temp))):
        job_mini = pr_1['hena_2_1_struct_mini_%s'%k]               # Bulk Crystal
    
        # Get last structure    
        struct_nvt = job_mini.get_structure(iteration_step=-1) # Last frame from minimization
    
        j = pr.create.job.Vasp("hena_1_crystal_small_%s_%sk"%(k,j), delete_existing_job=True)
        j.calc_md(temperature=temp[i], n_ionic_steps=25000, n_print=1, time_step=1.0)     # AIMD simulations        
        j.structure = struct_nvt                               # Intial structure   
        j.set_kpoints(scheme="GC")  
        j.input.kpoints.set_kpoints_file(size_of_mesh=['1 1 1'])
        j.set_exchange_correlation_functional("PBE")           # GGA (PBE) potential, also need to test SCAN, rSCAN, r2SCAN     

        #Select PAW potentials 
        #j.list_potentials()
        j.potential.Si = "Si"                                  # 4 valence electron, EMAX = 245 (eV)
        j.potential.P = "P"                                    # 5 valence electron, EMAX = 255 (eV) 
        j.potential.O = "O"                                    # 6 valence electron, EMAX = 400 (eV)   
        j.potential.Na = "Na"                                  # 7 valence electron with 'p' orbitals, EMAX = 260 (eV) 
        j.potential.Zr = "Zr_sv"                               # 12 valence electron with 's' orbitals, EMAX = 230 (eV)  

        # Edit INCAR file 
        j.input.incar["SMASS"] = 0                             # For noose hover algo     
        j.input.incar["MDALGO"] = 2                            # Nose-Hoover thermostat              
        j.input.incar["PREC"] = "Normal"
        j.input.incar["IBRION"] = 0                            # Conjugate gradient method, det. how the ions are updated and moved (remove if ions are not present)  
        j.input.incar["ISMEAR"] = 0                            # O = Gaussian, -1 = Fermi for AIMD 
        j.input.incar["LREAL"] = "A"                           # Projecion in real or reciprocal space   
        j.input.incar["SIGMA"] = 0.05                          # 0.03 or 0.05 for gaussian (1)  ----- Conv
        j.input.incar["EDIFF"] = 1e-5                          # Global break condition, accurate cal = 1e-8 (2) -- Conv 
        j.input.incar["ALGO"] = 'Normal'                       # For convergence use Fast else Normal
        # Paralellization 
        j.input.incar["ISYM"] = 0                              # no symmetry
        j.input.incar["MAXMIX"] = 40                           # reuse mixer from one MD step to next
        j.input.incar["NELMIN"] = 4                            # number of electronic steps
        j.input.incar["NCORE"] = 12                            # Core for calculations  
        #j.input.incar["KPAR"] = 12                            # No of K-points treated in parallel, must be divi to IBZKPT  
        j.server.cores = 192
        j.executable.executable_path = '/home/vd80naku/pyiron/resources/vasp/bin/run_vasp_6.1.2_default_mpi_gamma.sh'
        j.server.queue = "vasp_l_l2"
        j.run()